In [215]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from lxml import etree
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
import dateutil


In [216]:
#-------------------Extract-------------------------
urls = ["https://forecast.weather.gov/MapClick.php?lat=57.0826&lon=-135.2692#.Y-vs_9LMJkg",
        'https://forecast.weather.gov/MapClick.php?lat=45.5118&lon=-122.6756#.Y-vtHNLMJkg',
        'https://forecast.weather.gov/MapClick.php?lat=40.7143&lon=-74.006#.Y-vtQtLMJkg'
        ]
        
combined_df = pd.DataFrame()

for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.content,"html.parser")

    #various containers
    item1 = soup.find_all(id='current_conditions-summary')
    item2 = soup.find_all(id='current_conditions_detail')
    item3 = soup.find_all(id='tombstone-container')
    item4 = soup.find_all(id='tombstone-container')

    #raw data
    temp_f = [item.find(class_="myforecast-current-lrg").get_text() for item in item1]
    temp_min = soup.find('p', {'class': 'temp temp-low'}).text.strip()
    temp_max = soup.find('p', {'class': 'temp temp-high'}).text.strip()


    #df of temperatures
    df_temperature = pd.DataFrame({"temp" : temp_f,'tempmin': temp_min,'tempmax': temp_max})

    #df_2 is a df of current conditions in detail
    table = soup.find_all('table')
    df_2 = pd.read_html(str(table))[0]
    df_2 = df_2.pivot(columns=0, values=1).ffill().dropna().reset_index().drop(columns=['index'])

    #merge both dataframes
    temp_df=pd.concat([df_temperature,df_2],axis=1)

    #scrape lattitude, longitude, and elevation 
    lat_lon_elev = soup.find('span', {'class': 'smallTxt'}).text.strip()
    lat, lon, elev = re.findall(r'[-+]?\d*\.\d+|\d+', lat_lon_elev)

    #scrape name
    station = soup.find('h2', {'class': 'panel-title'}).text.strip()

    #add location, lat, long, and elev to source_df
    temp_df['elevation_ft'] = elev
    temp_df['latitude'] = lat
    temp_df['longitude'] = lon
    temp_df['weather_station'] = station

    combined_df = pd.concat([temp_df, combined_df], ignore_index=True, sort=False)

display(combined_df)



,temp,tempmin,tempmax,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,weather_station
0,51°F,Low: 43 °F,High: 59 °F,30.18 in (1021.1 mb),18°F (-8°C),27%,14 Feb 6:51 pm EST,10.00 mi,48°F (9°C),SW 8 mph,154,40.78,73.97,"New York City, Central Park (KNYC)"
1,45°F,Low: 27 °F,High: 45 °F,30.24 in (1024.04 mb),33°F (1°C),62%,14 Feb 03:53 PM PST,10.00 mi,NaN,N 0 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)"
2,37°F,Low: 28 °F,High: 31 °F,30.04 in (1017.2 mb),25°F (-4°C),62%,14 Feb 2:53 pm AKST,9.00 mi,28°F (-2°C),SE 14 G 22 mph,13,57.05,135.36,Sitka - Sitka Airport (PASI)


In [217]:
URLS = ['https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA',
        "https://www.wunderground.com/history/daily/us/or/portland",
        'https://www.wunderground.com/history/daily/us/ak/sitka/PASI'
        ]

precip_df = pd.DataFrame()

for URL in URLS:
    
    #create selenium web driver
    driver = webdriver.Chrome(r"C:/home/reed/.cache/selenium/chromedriver/linux64/109.0.5414.74")
    driver.get(URL)
    sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    #closes the pop up add if it appears otherwise continues to scrape the precip value from the page
    try:
        driver.find_element(By.XPATH,'/html/body/div[9]/div[3]/div/div/div/a').click()
    except:
        precip = driver.find_element(By.XPATH,"/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[2]/tr/td[1]").text

    df = pd.DataFrame([precip], columns=['precip'])
    precip_df = pd.concat([precip_df, df], ignore_index=True, sort=False)

    #quits the selenium driver
    driver.quit()

source_df =pd.concat([combined_df,precip_df],axis=1)
display(source_df)

/home/reed/Desktop/epicodus/team-week3/reed-work/venvteamweek/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # This is added back by InteractiveShellApp.init_path()


,temp,tempmin,tempmax,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,weather_station,precip
0,51°F,Low: 43 °F,High: 59 °F,30.18 in (1021.1 mb),18°F (-8°C),27%,14 Feb 6:51 pm EST,10.00 mi,48°F (9°C),SW 8 mph,154,40.78,73.97,"New York City, Central Park (KNYC)",0.00
1,45°F,Low: 27 °F,High: 45 °F,30.24 in (1024.04 mb),33°F (1°C),62%,14 Feb 03:53 PM PST,10.00 mi,NaN,N 0 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)",0.14
2,37°F,Low: 28 °F,High: 31 °F,30.04 in (1017.2 mb),25°F (-4°C),62%,14 Feb 2:53 pm AKST,9.00 mi,28°F (-2°C),SE 14 G 22 mph,13,57.05,135.36,Sitka - Sitka Airport (PASI),0.11


In [218]:
#-----------Data Transformations-----------------

# Convert 'lat' and 'lon' columns to float type
source_df[['latitude', 'longitude']] = source_df[['latitude', 'longitude']].astype(float)
# Convert 'elev' column to int type
source_df['elevation_ft'] = source_df['elevation_ft'].astype(int)
# Extract the numeric part of the temperature string and convert it to int
source_df['temp'] = source_df['temp'].str.extract('(\d+)').astype(int)
# Extract the numeric part of the tempmin string and convert it to int
source_df['tempmin'] = source_df['tempmin'].str.extract('(\d+)').astype(int)
# Extract the numeric part of the temperature string and convert it to int
source_df['tempmax'] = source_df['tempmax'].str.extract('(\d+)').astype(int)
# Split wind speed values into components and convert speed to int type
source_df['Wind Speed'] = source_df['Wind Speed'].str.extract('(\d+)', expand=False).fillna(0).astype(int)
# Convert 'humidity' column to int type
source_df['Humidity'] = source_df['Humidity'].str.extract('(\d+)', expand=False).astype(int)
# Convert 'barometer' column to float type, and convert inches to millibars
source_df['Barometer'] = round(source_df['Barometer'].apply(lambda x: float(x.split()[0]) * 33.8639 if 'in' in x and x != 'NA' else None), 2)
# Convert 'Visibility' column to float type
source_df['Visibility'] = source_df['Visibility'].str.extract('(\d+\.\d+|\d+)', expand=False).astype(float).round(2)
#Convert 'last_update' column to UTC
source_df['Last update'] = source_df['Last update'].apply(lambda x: dateutil.parser.parse(x, tzinfos={"EST": -5 * 3600, "CST": -6 * 3600, "MST": -7 * 3600,"PST": -8 * 3600,"AKST": -9 * 3600,"HST": -10 * 3600}))
source_df['Last update'] = source_df['Last update'].apply(lambda x: x.astimezone(dateutil.tz.tzutc()))
# make wind chill a float and only display degree F
source_df[['Wind Chill']] = source_df['Wind Chill'].str.extract('(\d+)', expand=True).astype(float)
# extract the numeric value of dewpoint and only display the degree n farenheit
source_df[['Dewpoint']] = source_df['Dewpoint'].str.extract('(\d+)', expand=True).astype(int)

display(source_df)
source_df.info()

,temp,tempmin,tempmax,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,weather_station,precip
0,51,43,59,1022.01,18,27,2023-02-14 23:51:00+00:00,10.0,48.0,8,154,40.78000,73.97000,"New York City, Central Park (KNYC)",0.00
1,45,27,45,1024.04,33,62,2023-02-14 23:53:00+00:00,10.0,NaN,0,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)",0.14
2,37,28,31,1017.27,25,62,2023-02-14 23:53:00+00:00,9.0,28.0,14,13,57.05000,135.36000,Sitka - Sitka Airport (PASI),0.11


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype                  
---  ------           --------------  -----                  
 0   temp             3 non-null      int64                  
 1   tempmin          3 non-null      int64                  
 2   tempmax          3 non-null      int64                  
 3   Barometer        3 non-null      float64                
 4   Dewpoint         3 non-null      int64                  
 5   Humidity         3 non-null      int64                  
 6   Last update      3 non-null      datetime64[ns, tzutc()]
 7   Visibility       3 non-null      float64                
 8   Wind Chill       2 non-null      float64                
 9   Wind Speed       3 non-null      int64                  
 10  elevation_ft     3 non-null      int64                  
 11  latitude         3 non-null      float64                
 12  longitude        3 non-nul

In [219]:

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import dateutil

# Split the 'location' column into separate 'city' and 'state' columns
df[['city', 'state']] = df['location'].str.split(', ', expand=True)

# Convert 'lat' and 'lon' columns to float type
df[['lat', 'lon']] = df[['lat', 'lon']].astype(float)

# Convert 'elev' column to int type
df['elev_ft'] = df['elev_ft'].astype(int)

# Extract the numeric part of the temperature string and convert it to int
df['temp_f'] = df['temperature'].str.extract('(\d+)').astype(int)

# Convert temperature to Celsius and add to new column 'temp_c'
df['temp_c'] = (df['temp_f'] - 32) * 5/9

# Round 'temp_c' to nearest integer and cast to int type
df['temp_c'] = df['temp_c'].round().astype(int)

# Convert 'humidity' column to float type
df['humidity'] = df['humidity'].str.extract('(\d+)', expand=False).astype(float) / 100

# Split wind speed values into components and convert speed to int type
df['wind_speed'] = df['wind_speed'].str.extract('(\d+)', expand=False).fillna(0).astype(int)

# Set any missing or non-numeric wind speed values to 0
df['wind_speed'] = df['wind_speed'].replace('Calm', 0)

# Convert 'barometer' column to float type, and convert inches to millibars
df['barometer'] = df['barometer'].apply(lambda x: float(x.split()[0]) * 33.8639 if 'in' in x and x != 'NA' else None)

# Round 'barometer' to two decimal places
df['barometer'] = df['barometer'].round(2)

# Split 'dewpoint' column into separate 'dewpoint_f' and 'dewpoint_c' columns
df[['dewpoint_f', 'dewpoint_c']] = df['dewpoint'].str.extract('(\d+).*?(\d+)', expand=True).astype(int)

# Convert 'vis_miles' column to float type
df['vis_miles'] = df['vis_miles'].str.extract('(\d+\.\d+|\d+)', expand=False).astype(float).round(2)

# Split 'wind_chill' column into separate 'wind_chill_f' and 'wind_chill_c' columns
df[['wind_chill_f', 'wind_chill_c']] = df['wind_chill'].str.extract('(\d+).*?(\d+)', expand=True).astype(float)

# Convert 'last_update' column to datetime type with the desired format and time zone
df['last_update'] = df['last_update'].apply(lambda x: dateutil.parser.parse(x, tzinfos={'CST': dateutil.tz.tzoffset(None, -21600)}))

# Convert 'last_update' column to UTC
df['last_update'] = df['last_update'].apply(lambda x: x.astimezone(dateutil.tz.tzutc()))

# Drop columns that were split into two values
df = df.drop(['temperature', 'dewpoint', 'wind_chill'], axis=1)

# Move 'city' and 'state' columns after 'location' column
city = df.pop('city')
state = df.pop('state')
temp_f = df.pop('temp_f')
temp_c = df.pop('temp_c')
df.insert(1, 'city', city)
df.insert(2, 'state', state)
df.insert(6, 'temp_f', temp_f)
df.insert(7, 'temp_c', temp_c)

# Move 'last_update' column to last position
last_update = df.pop('last_update')
df['last_update'] = last_update


print(df)

KeyError: 'location'